In [2]:
import numpy as np
import h5py
from matplotlib import pyplot as plt
import yaml
import torch
import torchvision
import sys
import json
import os
from os.path import join
import glob
sys.path.append('../')


In [18]:
a = torch.tensor([[1,2], [3,4], [5,6]])
b = torch.tensor([[7,8], [1,8]])
feats = [a, b]
num_obj = [3,2]

num_rel = [no*no for no in num_obj]  # num_rel = R
mask = 1-torch.cat([torch.eye(no).view(-1) for no in num_obj])  # BR

rel_inp = torch.cat([torch.cat([ft.repeat_interleave(no, 0), ft.repeat(no, 1)], 1) for ft, no in zip(feats, num_obj)], 0)  # BR x 2d
print(rel_inp)
effects = rel_inp  # BR x h
effects = effects * mask[:, None]
effects = effects.split(num_rel, dim=0)  # B x [R x h]
print(effects)
effects = [eff.view(no, no, -1).sum(1) for eff, no in zip(effects, num_obj)]  # B x [O x h]
print(effects)


tensor([[1, 2, 1, 2],
        [1, 2, 3, 4],
        [1, 2, 5, 6],
        [3, 4, 1, 2],
        [3, 4, 3, 4],
        [3, 4, 5, 6],
        [5, 6, 1, 2],
        [5, 6, 3, 4],
        [5, 6, 5, 6],
        [7, 8, 7, 8],
        [7, 8, 1, 8],
        [1, 8, 7, 8],
        [1, 8, 1, 8]])
(tensor([[0., 0., 0., 0.],
        [1., 2., 3., 4.],
        [1., 2., 5., 6.],
        [3., 4., 1., 2.],
        [0., 0., 0., 0.],
        [3., 4., 5., 6.],
        [5., 6., 1., 2.],
        [5., 6., 3., 4.],
        [0., 0., 0., 0.]]), tensor([[0., 0., 0., 0.],
        [7., 8., 1., 8.],
        [1., 8., 7., 8.],
        [0., 0., 0., 0.]]))
[tensor([[ 2.,  4.,  8., 10.],
        [ 6.,  8.,  6.,  8.],
        [10., 12.,  4.,  6.]]), tensor([[7., 8., 1., 8.],
        [1., 8., 7., 8.]])]


In [8]:
from utils.cpconfig import update_to_ccn2_paths
from utils.train_utils import get_readout_dataloader
from dataset.data_utils import SCENARIOS

In [18]:
cfg = yaml.safe_load("""
data:
  dataset_class: object_feats_readout
  physion_path: '/mnt/fs3/rmvenkat/data/physion_train_data/'
  saved_data_path: /mnt/fs5/chpatel/saved_data/dataset
  dynamic_feat_dir:  /ccn2/u/rmvenkat/chpatel/test/gnn_aefullneg_hlr_all/feats/
  scenario : 'Dominoes'
  split_percentage: 0.9

model:
  model_name: linear_readout
  ckpt_path: ''
  feat_size: 128
  num_pointers: 8  # used in graph readout
  max_num_obj: -1  # It will be decided automatically
  nonlinearity: 'sigmoid'
  num_frames: -1  # It will be decided by readout mode and other parameters
  readout_mode: 'full'  # ['obs_sim', 'obs', 'full']
  seed_len: 15
  unroll: 45
""")
cfg = update_to_ccn2_paths(cfg)

In [23]:
for scen in SCENARIOS:
    cfg['data']['scenario'] = scen
    ds = get_readout_dataloader(data_config=cfg['data'])
    max_obj = 0
    for s in ds:
        max_obj = max(s['obs_sim_feats'][0].shape[1], max_obj)
    print(scen, max_obj)

Collide None Dataset size 1158
Collide 7
Dominoes None Dataset size 1146
Dominoes 11
Drop None Dataset size 1150
Drop 7
Roll None Dataset size 1149
Roll 7
Contain None Dataset size 1109
Contain 10
Drape None Dataset size 1149
Drape 6
Link None Dataset size 1103
Link 15
Support None Dataset size 1093
Support 9


In [35]:
x = torch.tensor([0, 5, 30, 40]).long()

In [36]:
a = x[1:] - x[:-1]

In [38]:
b

tensor([2.6667, 2.6667, 2.6667, 2.6667, 2.6667, 0.5333, 0.5333, 0.5333, 0.5333,
        0.5333, 0.5333, 0.5333, 0.5333, 0.5333, 0.5333, 0.5333, 0.5333, 0.5333,
        0.5333, 0.5333, 0.5333, 0.5333, 0.5333, 0.5333, 0.5333, 0.5333, 0.5333,
        0.5333, 0.5333, 0.5333, 1.3333, 1.3333, 1.3333, 1.3333, 1.3333, 1.3333,
        1.3333, 1.3333, 1.3333, 1.3333])

In [33]:
a

tensor([ 5, 10,  5])